In [ ]:
from datetime import datetime
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import db

%matplotlib inline

In [ ]:
engine = db.get_engine()

In [ ]:
table_name = 'm1m3_HardpointActuatorData'
expected_rate = 50  # Hz

In [ ]:
def make_delta_time(df_col, dtype=None):
    ts = df_col.values
    dt = ts[1:] - ts[:-1]
    if dtype is None:
        return ts, dt
    else:
        return ts.astype(dtype), dt.astype(dtype)

In [ ]:
b1 = datetime(2019, 1, 23, 22, 25, 0)
b2 = datetime(2019, 1, 23, 22, 35, 0)
dt = b2 - b1
expected_num_messages = dt.seconds * expected_rate

In [ ]:
query = 'select date_time,private_sndStamp,private_rcvStamp from {0} where {0}.date_time between \'{1}\' and \'{2}\';'.format(table_name, b1, b2)
print(query)

In [ ]:
df1 = pd.read_sql_query(query, engine)

In [ ]:
delta_messages = expected_num_messages - df1.shape[0]
print('Expected {} messages, got {} messages, missing {} messages'.format(expected_num_messages, df1.shape[0], delta_messages))

In [ ]:
insert_times = df1['date_time'].values
send_times = df1['private_sndStamp'].values
recv_times = df1['private_rcvStamp'].values
for i in range(10):
    print("{:.10f}\t{:.10f}\t{:.10f}".format(float(insert_times[i])/1e9, send_times[i], recv_times[i]))

print(send_times[:5] - insert_times[:5].astype('float')/1e9)

In [ ]:
insert_time, dt_insert_ns = make_delta_time(df1['date_time'], dtype=np.int64)
dt_insert = dt_insert_ns / 1e9
send_time, dt_send = make_delta_time(df1['private_sndStamp'])
recv_time, dt_recv = make_delta_time(df1['private_rcvStamp'])

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(10, 5))
ax[0].hist(dt_send, bins=100)
ax[0].set_title('Send Time')
ax[0].set_xlabel('$\Delta T$ (s)')
ax[0].set_yscale('log')
ax[1].hist(dt_recv, bins=100)
ax[1].set_title('Receive Time')
ax[1].set_xlabel('$\Delta T$ (s)')
ax[1].set_yscale('log')
ax[2].hist(dt_insert, bins=100)
ax[2].set_title('Insert Time')
ax[2].set_xlabel('$\Delta T$ (s)')
ax[2].set_yscale('log')
fig.subplots_adjust(wspace=0.2)

In [ ]:
plt.scatter(dt_send, dt_recv)
plt.xlabel("$\Delta$ Send Time")
plt.ylabel('$\Delta$ Receive Time')

In [ ]:
long_time = dt_send > 0.03

In [ ]:
orig_ts = (send_time[:-1][long_time]).astype('datetime64[s]')

In [ ]:
orig_ts

In [ ]:
print(send_time.dtype)
indexes = np.where(dt_send > 0.03)
#print(indexes[0])
print((send_time[:-1][indexes] * 1e9).astype('datetime64[ns]'))
next_indexes = indexes[0] + 1
print((send_time[:-1][next_indexes] * 1e9).astype('datetime64[ns]'))
print(dt_send[indexes])

In [ ]:
insert_time = df1['date_time'].values

In [ ]:
insert_long = insert_time[:-1][long_time]

In [ ]:
insert_long

In [ ]:
datetime.utcfromtimestamp(1547155885.310383)

In [ ]:
(770366976-730547968)/1e9